In [1]:
import numpy as np
import darknet
import cv2
import time 

In [2]:
# #다크넷에 cfg라는 모델층과 기존 학습된 weight를 연결
# net = darknet.load_net_custom("./go.cfg".encode('utf-8'),"./go.weights".encode('utf-8'), 0,1)
# # 데이터도 연결 / coco.data안에는 80개의 클래스를 알아볼수있는 이름들이 있는데 그중에 car도 잇음
# meta = darknet.load_meta("./go.data".encode('utf-8'))

#다크넷에 cfg라는 모델층과 기존 학습된 weight를 연결
net = darknet.load_net_custom("./final.cfg".encode('utf-8'),"./final.weights".encode('utf-8'), 0,1)
# 데이터도 연결 / coco.data안에는 80개의 클래스를 알아볼수있는 이름들이 있는데 그중에 car도 잇음
meta = darknet.load_meta("./go.data".encode('utf-8'))


In [3]:
#그림 불러오기
#여러개 불러올려면 당연히 for문해야함
def start_detect():
    #for name in sorted(os.listdir(dir)[-1:]):
    names = sorted(os.listdir(dir))
    try:
        for name in names: # 배치를 전부 돌리려면 풀어줘야 함.
            if name =='.ipynb_checkpoints':
                continue
            cctv_name = name[:-4].split('_')[1:]
            cctv_lay_down = [0]*len(cctv_name)

            a = cv2.imread(dir+name)
            detections = darknet.detect_image(net, meta, a, thresh=0.3)

            for detection in detections:
                if detection[0]==b'lay_down':
                    x,y=map (int,detection[2][0:2])            
                    # print(heat_map[x,y]) heat_map[x,y] # 이게 lay_down의 좌표가 된다.
                    cctv_lay_down[heat_map[x,y]]=1 # 원래라면 cctv_lay_down의 개수와 창 크기에 맞춰서 heat_map을 재편할 수 있도록 해야함. 그러나 그건 차후의 과제
                    #print(cctv_lay_down)

            for i in range(len(cctv_name)):
                dict_[cctv_name[i]] = dict_.get(cctv_name[i],deque([0]*9)) # 처음이라면 큐 자료형 만들어줌
                if cctv_lay_down[i]==1:
                    dict_[cctv_name[i]].append(1)
                    if sum(dict_[cctv_name[i]]) >= 10: #10초간 10번 감지되었을때
                        if cctv_name[i] not in cool_time:
                            # check_victim(cctv_name[i])
                            #print(cctv_name[i],'의식확인 실행')
                            f = open('check_cctv/'+cctv_name[i]+'.txt','w')
                            f.close()
                            cool_time[cctv_name[i]] = 300 # 쿨타임
                            dict_[cctv_name[i]] = deque([0]*10)
                        #else: 
                            #print(cctv_name[i],cool_time[cctv_name[i]],'초 의식확인 쿨타임 적용')
                else:
                    dict_[cctv_name[i]].append(0)
                dict_[cctv_name[i]].popleft()

            # 확인이 끝난 사진을 삭제
            os.remove(dir+name)
    
    except:
        #print('데이터 없음')
        pass

In [6]:
import os
from collections import deque
import numpy as np

dir ='4windows_input/'
window = [240,240] # 1/4 윈도우의 너비와 높이( 전체 말고 각자) # 총 480이 된다.
heat_map = np.array( [[0]*window[0] + [1]*window[0]] * window[1]
                    +[[2]*window[0] + [3]*window[0]] * window[1])
cool_time = {}
dict_ = {}

for _ in range(100000): # 데이터는 40개
    start_detect()
    print(f'\rYolov4 동작인식 cctv:\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t{dict_}\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t감지된 응급 상황 발생 cctv 및 경과시간: {cool_time}                                                                         ',end='')

    
    del_list = []
    for cool in cool_time:
        cool_time[cool] -=1
        if cool_time[cool] < 1:
          del_list.append(cool)
    for del_ in del_list:
        del cool_time[del_]
    
    time.sleep(1)


Yolov4 동작인식 cctv:																{'골목': deque([0, 1, 0, 1, 0, 1, 0, 1, 0]), '합중문고': deque([1, 0, 1, 0, 1, 0, 1, 0, 1]), '강서구코너': deque([0, 0, 0, 0, 0, 0, 0, 0, 0]), '집앞': deque([0, 0, 0, 0, 0, 0, 0, 0, 0]), '삼성부동산': deque([0, 0, 0, 0, 0, 0, 0, 0, 0]), '횡단보도': deque([0, 0, 0, 0, 0, 0, 0, 0, 0]), '버스정류소': deque([0, 0, 0, 0, 0, 0, 0, 0, 0]), '길거리': deque([0, 0, 0, 0, 0, 0, 0, 0, 0]), '현관': deque([1, 1, 1, 1, 1, 1, 1, 1, 1]), '베란다': deque([0, 0, 0, 0, 0, 0, 0, 0, 0]), '거실': deque([0, 0, 0, 0, 0, 0, 0, 0, 0]), '주방': deque([0, 0, 0, 0, 0, 0, 0, 0, 0])}																								감지된 응급 상황 발생 cctv 및 경과시간: {'현관': 252}                                                                         

KeyboardInterrupt: 